In [1]:
import tensorflow as tf
from tensorflow.python.platform import gfile
from show_graph import show_graph
import time
import cv2
import numpy as np
from keras.preprocessing import image

img_height = 300 # Height of the input images
img_width = 480 # Width of the input images
COLORS = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
FONT = cv2.FONT_HERSHEY_SIMPLEX

classes = ['background', 'car', 'truck', 'pedestrian', 'bicyclist', 'light']

sess=tf.InteractiveSession()
f = gfile.FastGFile("model/mnet_fast_inference.pb", 'rb')
graph_def = tf.GraphDef()
# Parses a serialized binary message into the current message.
graph_def.ParseFromString(f.read())
f.close()

sess.graph.as_default()
# Import a serialized TensorFlow `GraphDef` protocol buffer
# and place into the current default `Graph`.
tf.import_graph_def(graph_def)
show_graph(tf.get_default_graph().as_graph_def())


Using TensorFlow backend.


Instructions for updating:
Use tf.gfile.GFile.


In [2]:
for op in sess.graph.get_operations():
    print(op.name, op.type)

import/input_1 Placeholder
import/confidence_thresh Const
import/top_k Const
import/normalize_coords Const
import/img_height Const
import/img_width Const
import/nms_max_output_size Const
import/conv2d_1/kernel Const
import/conv2d_1/kernel/read Identity
import/conv2d_1/bias Const
import/conv2d_1/bias/read Identity
import/conv2d_1/convolution Conv2D
import/conv2d_1/BiasAdd BiasAdd
import/re_lu_1/Relu6 Relu6
import/conv2d_2/kernel Const
import/conv2d_2/kernel/read Identity
import/conv2d_2/bias Const
import/conv2d_2/bias/read Identity
import/conv2d_2/convolution Conv2D
import/conv2d_2/BiasAdd BiasAdd
import/re_lu_2/Relu6 Relu6
import/depthwise_conv2d_1/depthwise_kernel Const
import/depthwise_conv2d_1/depthwise_kernel/read Identity
import/depthwise_conv2d_1/bias Const
import/depthwise_conv2d_1/bias/read Identity
import/depthwise_conv2d_1/depthwise DepthwiseConv2dNative
import/depthwise_conv2d_1/BiasAdd BiasAdd
import/re_lu_3/Relu6 Relu6
import/conv2d_3/kernel Const
import/conv2d_3/kernel/re

import/decoded_predictions/loop_over_batch/while/Enter_1 Enter
import/decoded_predictions/loop_over_batch/while/Enter_2 Enter
import/decoded_predictions/loop_over_batch/while/Merge Merge
import/decoded_predictions/loop_over_batch/while/Merge_1 Merge
import/decoded_predictions/loop_over_batch/while/Merge_2 Merge
import/decoded_predictions/loop_over_batch/while/Less/Enter Enter
import/decoded_predictions/loop_over_batch/while/Less Less
import/decoded_predictions/loop_over_batch/while/Less_1 Less
import/decoded_predictions/loop_over_batch/while/LogicalAnd LogicalAnd
import/decoded_predictions/loop_over_batch/while/LoopCond LoopCond
import/decoded_predictions/loop_over_batch/while/Switch Switch
import/decoded_predictions/loop_over_batch/while/Switch_1 Switch
import/decoded_predictions/loop_over_batch/while/Switch_2 Switch
import/decoded_predictions/loop_over_batch/while/Identity Identity
import/decoded_predictions/loop_over_batch/while/Identity_1 Identity
import/decoded_predictions/loop_ov

In [3]:
pred_tensor = sess.graph.get_tensor_by_name('import/decoded_predictions/loop_over_batch/TensorArrayStack/TensorArrayGatherV3:0')

In [4]:

def pre_pro_img(frame):
    orig_images = [] # Store the images here.
    input_images = [] # Store resized versions of the images here.
    orig_images.append(frame)
    #img = image.load_img(frame, target_size=(img_height, img_width))
    img = cv2.resize(orig_images[0], (480, 300))
    img = image.img_to_array(img)
    input_images.append(img)
    input_images = np.array(input_images)
    
    return orig_images, input_images

In [5]:
def predict(orig_images, input_images):
    y_pred = sess.run(pred_tensor, {'import/input_1:0': input_images})
    
    confidence_threshold = 0.6
    y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]
    np.set_printoptions(precision=2, suppress=True, linewidth=90)
    
    for box in y_pred_thresh[0]:
        # Transform the predicted bounding boxes for the 300x300 image to the original image dimensions.
        xmin = int(box[-4] * orig_images[0].shape[1] / img_width)
        ymin = int(box[-3] * orig_images[0].shape[0] / img_height)
        xmax = int(box[-2] * orig_images[0].shape[1] / img_width)
        ymax = int(box[-1] * orig_images[0].shape[0] / img_height)
        color = COLORS[int(box[0]) % 3]
        label = '{}: {:.2f}'.format(classes[int(box[0])], box[1])
        cv2.rectangle(orig_images[0], (xmin, ymin), (xmax, ymax), color, 2)
        
        text_top = (xmin, ymin-10)
        text_bot = (xmin + 80, ymin + 5)
        text_pos = (xmin + 5, ymin)
        cv2.rectangle(orig_images[0], text_top, text_bot, color, -1)
        cv2.putText(orig_images[0], label, text_pos, FONT, 0.35, (0,0,0), 1)
    return orig_images[0]

In [6]:

cap = cv2.VideoCapture('examples/sample1.mp4')

while(True):
    ret, frame = cap.read()
    start_time = time.time()
    orig_images, input_images = pre_pro_img(frame)
    pred_frame = predict(orig_images, input_images)
    print(time.time() - start_time)
    cv2.imshow('frame', pred_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1.8751859664916992
0.015659570693969727
0.022940397262573242
0.015620946884155273
0.035286664962768555
0.015622377395629883
0.03124260902404785
0.02592945098876953
0.010416507720947266
0.015619516372680664
0.031243562698364258
0.027935504913330078
0.016785860061645508
0.031245708465576172
0.020941972732543945
0.019106626510620117
0.015620946884155273
0.015620708465576172
0.02193927764892578
0.021941661834716797
0.016813039779663086
0.02094292640686035
0.011308670043945312
0.031243324279785156
0.015620946884155273
0.015618324279785156
0.036570072174072266
0.011102914810180664
0.021997451782226562
0.020943403244018555
0.020943164825439453
0.02094268798828125
0.02116107940673828
0.022939205169677734
0.020943403244018555
0.021942138671875
0.022937536239624023
0.024932384490966797
0.02792525291442871
0.021941423416137695
0.02293872833251953
0.022938013076782227
0.02293539047241211
0.0249330997467041
0.02393507957458496
0.021940946578979492
0.02194070816040039
0.02094292640686035
0.020944356

0.02094411849975586
0.019944190979003906
0.010874748229980469
0.015570878982543945
0.031243324279785156
0.015622377395629883
0.015619754791259766
0.031243085861206055
0.0229341983795166
0.01725935935974121
0.015622854232788086
0.015623331069946289
0.031241416931152344
0.015622854232788086
0.015620231628417969
0.03623032569885254
0.02194046974182129
0.015626192092895508
0.031236648559570312
0.01562190055847168
0.015620946884155273
0.031241655349731445
0.015623092651367188
0.017934799194335938
0.020940303802490234
0.019167661666870117
0.01506185531616211
0.031242847442626953
0.015624761581420898
0.015619993209838867
0.031243324279785156
0.015622615814208984
0.022588729858398438
0.01994609832763672
0.015621662139892578
0.03449082374572754
0.01891016960144043
0.015621662139892578
0.02302718162536621
0.016083955764770508
0.021935701370239258
0.0062444210052490234
0.015619277954101562
0.031247854232788086
0.015618324279785156
0.015618562698364258
